In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/media/beans/ssd/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import run_epoch, eval_rw

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'3nprtb0a'

In [3]:
wandb.init(id='s6nia8eb', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

13332.809

In [5]:
m.backbone

EfficientNet(
  (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
        (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
 

In [6]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [7]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=1e-4)

In [8]:
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [9]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


In [10]:
#wandb.watch(m, log="all")

In [11]:
model_stem = "6.19"
START_E = 0

In [ ]:
n_epochs = 100
log_wandb = True

for e in range(START_E, n_epochs):
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2650)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.00205354, 'consistency losses/trn_steer_cost': 0.00204248, 'consistency losses/trn_te_loss': 0.00761967, 'consistency losses/trn_torque_loss': 138.65978774, 'consistency losses/trn_torque_delta_loss': 48.98364258, 'logistical/obs_consumed_per_second': 243.78740157, 'logistical/obs_generated_per_second': 117.22047244, 'logistical/data_consumption_ratio': 2.08060263, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.23593903, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 238.921875, 'logistical/obs_generated_per_second': 118.2109375, 'logistical/data_consumption_ratio': 2.0277708, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00267693, 'consistency losses/trn_steer_cost': 0.00272791, 'consistency losses/trn_te_loss': 0.01010852, 'consistency losses/trn_torque_loss': 208.82738095, 'consistency losses/trn_torque_delta_loss': 49.46875, 'logistical/max_param': 16.23698044, 'logistical/lr': 0.0001}
{'logistical/obs

{'logistical/obs_consumed_per_second': 237.2265625, 'logistical/obs_generated_per_second': 115.9921875, 'logistical/data_consumption_ratio': 2.04511007, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00207867, 'consistency losses/trn_steer_cost': 0.00200373, 'consistency losses/trn_te_loss': 0.00746831, 'consistency losses/trn_torque_loss': 162.39598881, 'consistency losses/trn_torque_delta_loss': 44.23529053, 'logistical/max_param': 16.24037743, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 238.359375, 'logistical/obs_generated_per_second': 116.2890625, 'logistical/data_consumption_ratio': 2.04975275, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00234307, 'consistency losses/trn_steer_cost': 0.0023653, 'consistency losses/trn_te_loss': 0.00884393, 'consistency losses/trn_torque_loss': 190.96104754, 'consistency losses/trn_torque_delta_loss': 54.14477539, 'logistical/max_param': 16.23885155, 'logistical/lr': 0.0001}
{'logistical/obs_consume

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s


In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e16.torch")